# =========================================
# Prototype B – With Generative AI
# =========================================

# -----------------------------
### 1. Import libraries
# -----------------------------

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
import pickle, os


# -----------------------------
### 2. Load data
# -----------------------------

In [ ]:

df = pd.read_csv("../data/2022_Q1_OR_Utilization.csv")
df['Date'] = pd.to_datetime(df['Date'], format="%m/%d/%y")
for col in ['Wheels In','Start Time','End Time','Wheels Out']:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Creating the columns procedure_minutes, setup_minutes, exit_minutes
df['procedure_minutes'] = (df['End Time'] - df['Start Time']).dt.total_seconds()/60
df['setup_minutes'] = (df['Start Time'] - df['Wheels In']).dt.total_seconds()/60
df['exit_minutes']  = (df['Wheels Out'] - df['End Time']).dt.total_seconds()/60

# Fiiltering the dataset based on  hourly basis
df = df[(df['procedure_minutes'] > 0) & (df['procedure_minutes'] < 360)]

# Find the shape of the dataset
print(df.shape)

(2172, 16)


/var/folders/gl/rtk4gdps67v0p982cpbqjhs80000gn/T/ipykernel_5870/1977921279.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
/var/folders/gl/rtk4gdps67v0p982cpbqjhs80000gn/T/ipykernel_5870/1977921279.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
/var/folders/gl/rtk4gdps67v0p982cpbqjhs80000gn/T/ipykernel_5870/1977921279.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
/var/folders/gl/rtk4gdps67v0p982cpbqjhs80000gn